Testnet here: https://testnet.binancefuture.com/en/futures/BTCUSDT

In [1]:
api_key = "6ce63f3406fd8ebbff01054a66c25fe3c851c45932088c8ca3131a7005188462"
secret_key = "aa3ea32929252467fa5ffeac5818c95beabfb5dba691ef445e7eaa31ea0d15f6"

In [2]:
from lib.TechnicalIndicators import *
from binance.client import Client
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

2022-12-21 02:57:42.584294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 02:57:43.238627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-21 02:57:43.238682: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-21 02:57:43.359255: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-21 02:57:46.023377: W tensorflow/stream_executor/platform/default

In [3]:
class FuturesTrader():
    def __init__(self, symbol="btcusd", testnet = True):
        ####API CONNECTIONS ####
        self.stream = None
        self.client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = testnet)
        #######################
        self.data = None
        self.symbol = symbol
        self.asset = self.get_asset(self.symbol) #get asset like "USDT"
        #### "BTCUSD" is not valid with official api methods, need to use "BTCUSDT" ####
        symbol = self.symbol + "t" if self.symbol.endswith('usd') else self.symbol
        self.symbol_upper = symbol.upper()
        self.strategies = [] #this stores the strategies used
        self.open_orders = []
        self.leverage = 1 #stores current leverage
        self.current_pos = 0 #stores current position
        self.ind_sum = 0 #stores indicators sum
        self.initial_balance = self.get_current_balance() #sotres the initial balance of the session
        self.available_balance = self.initial_balance #stores available balance
        self.last_close_price = 0 #stores last close price
        self.weights_sum = 0 #keeps weight sum of strategies to use it with leverage
        self.rnn = None #stores rnn
        self.bound = 0.6 #stores bound of strategy for rnn
        self.weights = [0.08877842031603478,
  0.05987706928852676,
  0.13120415318448939,
  0.023081971180099897,
  1.2296684936304951,
  0.05058368745150239,
  0.027529078706913866,
  0.017268540027698426,
  0.12069770802559819,
  0.08091687035480966]
    
    def get_asset(self, symbol):
        if symbol.endswith('busd'): return "BUSD"
        if symbol.endswith('usd'): return "USDT"
        if symbol.endswith('eth'): return "ETH"
        if symbol.endswith('bnb'): return "BNB"
        if symbol.endswith('btc'): return "BTC"
        
    def message_handler(self, msg):
        if 'result' in msg.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        
        QAVol   = float(msg["k"]["q"])
        NoT     = float(msg["k"]["n"])
        TBBAV   = float(msg["k"]["V"])
        TBQAV   = float(msg["k"]["Q"])
        
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        col_num = self.data.shape[1]
        self.data.loc[start_time] = [first, high, low, close, volume,
                                     QAVol, NoT, TBBAV, TBQAV, complete] + [False]*(col_num-10)
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.last_close_price = close
            #print("candle completed", end="")
            self.run_strategy()
        
    def start_streaming(self, interval="1m"):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol=self.symbol.lower() +"_perp",
            id=2,
            interval=interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()
        
    def get_most_recent_data(self, num_candles=100, interval = "1m"):
        #### Get start time for candles ####
        now = datetime.utcnow()
        past = str(now - self.available_intervals(num_candles)[interval])
        #### Request candles and prepare the df ####
        bars = self.client.futures_historical_klines(symbol = self.symbol_upper.lower(), 
                                        interval = interval, 
                                        start_str =past,
                                        end_str = None)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Close Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        use_columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Quote Asset Volume",
                       "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
        df = df[use_columns].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]    
        self.data = df
    
    def start_trading(self, num_candles = 100, interval = "1m"):
        if interval in self.available_intervals(num_candles).keys():
            self.get_most_recent_data(num_candles = num_candles, interval=interval)
            self.prepare_strategies()
            self.start_streaming(interval)
        else:
            print("That interval is not available")
            
    def stop_trading(self):
        self.stop_streaming()
        self.go_neutral()
        #print ending metrics here!!
        
    def available_intervals(self, candles_required):
        '''
        Helper function for "get_most_recent_data" method.
        
        '''
        return {
            "1m"  : timedelta(minutes=candles_required),
            "3m"  : timedelta(minutes=candles_required*3),
            "5m"  : timedelta(minutes=candles_required*5),
            "15m" : timedelta(minutes=candles_required*15),
            "30m" : timedelta(minutes=candles_required*30),
            "1h"  : timedelta(hours=candles_required),
            "2h"  : timedelta(hours=candles_required*2),
            "4h"  : timedelta(hours=candles_required*4),
            "6h"  : timedelta(hours=candles_required*6),
            "8h"  : timedelta(hours=candles_required*8),
            "12h" : timedelta(hours=candles_required*12),
            "1d"  : timedelta(days=candles_required),
            "3d"  : timedelta(days=candles_required*3),
            "1w"  : timedelta(days=candles_required*7),
            "1M"  : timedelta(days=candles_required*28) #this may give less than the desired candles because each month has different amount of days
        }
    def prepare_strategies(self):
        # prepare params
        #SMA
        SMA_S = 5
        SMA_L = 13
        #EWMA
        approx_avg_period_s = 5
        approx_avg_period_l = 8
        #BBS
        dev = 1.5
        periods = 10
        #prepare strategies
        self.strategies = [
            SMA(
                data = self.data,
                SMA_S = SMA_S,
                SMA_L = SMA_L,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[0]
            ),
            EWMA(
                data = self.data,
                approx_avg_period_s = approx_avg_period_s,
                approx_avg_period_l = approx_avg_period_l,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[1]
            ),
            BollingerBands(
                data = self.data,
                dev = dev, 
                periods = periods,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[2]
            ),
            MACD(
                data = self.data,
                fast =12,
                slow =26, 
                signal = 9,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[3]
            ),
            RSI(
                data = self.data,
                length = 14,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[4]
            ),
            Hammer(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[5]
            ),
            Doji(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[6]
            ),
            EBSW(
                data = self.data,
                default_strategy = 1,
                column = "Close",
                weight = self.weights[7]
            ),
            ADX(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[8],
            ),
            KVO(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[9],
            ),
            OBV(
                data = self.data
            )
        ]
        for strategy in self.strategies:
            strategy.calculate() #add columns to data 
        #self.data.dropna(inplace = True) #dropna after calculating the strategy
        self.rnn = RNN(
            data = self.data,
            default_strategy = 1,
            weight = 1,
            model = 'models/rnn_15.h5',
            scaler = 'models/scaler_dic19_1m.pkl'
        )
        self.rnn.calculate() 
        #self.calculate_weights_sum()
    
    def leverage_strategy(self, ind_sum, ind_count, min_lev = 5, max_lev = 15):
        if max_lev < min_lev:
            print("max leverage is less than min leverage")
            return -1
        #y2 = y1 + m(x2-x1) , m =(y2-y1)/(x2-x1)
        #two points of the line: (ind_sum = 0.5, min_lev) and (ind_sum = self.weights_sum, max_lev)
        ind_sum = abs(ind_sum) #short or long, we want just abs number
        m = (max_lev-min_lev)/(1-self.bound)
        new_leverage = round( min_lev + m*( ind_sum - self.bound ))
        self.client.futures_change_leverage(symbol = self.symbol_upper, leverage = new_leverage)
        self.leverage = new_leverage
        return new_leverage
    
    def run_strategy(self):
        self.ind_sum = 0
        for strategy in self.strategies:
            strategy.calculate_for_last_row()
        #    self.ind_sum += strategy.strategy(-1) * strategy.weight
        #if abs(self.ind_sum) < 0.5: self.ind_sum = 0 #discards weak signals < |0.5|    
        #self.predicted_pos = np.sign(self.ind_sum)
        self.rnn.calculate_for_last_row()
        self.predicted_pos, self.ind_sum = self.rnn.strategy(-1, bound = self.bound) 
        print(" |pp:"+str(self.predicted_pos)+str("| "), end="")
        if self.predicted_pos == 0:
            self.go_neutral()
        elif self.predicted_pos == 1 and self.get_position() in [0, -1]:
            self.go_long(prc = True, amount = 95) # go long with full amount
        elif self.predicted_pos == -1 and self.get_position() in [0, 1]:
            self.go_short(prc = True, amount = 95) # go short with full amount
            
    def go_long(self, prc = True, amount = None):
        if self.get_position() != 0:
            self.go_neutral() #if some position, go neutral first
        if self.should_end_session(): return
        self.leverage_strategy(ind_sum = self.ind_sum, ind_count = len(self.strategies))
        if prc == True: 
            amount = (self.available_balance * amount/100) * self.leverage
        else:
            amount = amount * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  round(amount/self.last_close_price, 3)
        self.create_order(side = "BUY", quantity = quantity)

    def go_short(self, prc = True, amount = None):
        if self.get_position() != 0:
            self.go_neutral() #if some position, go neutral first
        if self.should_end_session(): return
        self.leverage_strategy(ind_sum = self.ind_sum, ind_count = len(self.strategies))
        if prc == True: 
            amount = (self.available_balance * amount/100) * self.leverage
        else:
            amount = amount * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  round(amount/self.last_close_price, 3)
        self.create_order(side = "SELL", quantity = quantity)
    
    def go_neutral(self):
        if self.get_position() == 1: #if long, sell all
            self.create_order(side = "SELL", close_pos = True)
        elif self.get_position() == -1: #if short, buy all
            self.create_order(side = "BUY", close_pos = True)
    
    def create_order(self, side = "BUY", quantity = 0, close_pos = False):
        if close_pos:
            open_order = self.open_orders[0]
            quantity = open_order["executedQty"]
            order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity)
            order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
            self.open_orders = []
            self.current_pos = 0
            self.available_balance = self.get_current_balance()
            print(" |CP| ", end="")
            return 0
        order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity)
        order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
        self.open_orders.append(order)
        self.current_pos = 1 if side == "BUY" else -1
        print(" |OP| ", end="")
        return 0
    
    def get_position(self):
        return self.current_pos
    
    def should_end_session(self):
        if self.available_balance < self.initial_balance * 0.3 and self.get_position() == 0: #no money and no positions
            self.stop_trading()
            return True
    def get_current_balance(self):
        balance = pd.DataFrame(self.client.futures_account_balance())# Asset Balance details
        balance = float(balance[ balance["asset"] == self.asset ].iloc[0]["balance"])
        return balance
    
    def calculate_weights_sum(self):
        self.weights_sum = 0
        for strategy in self.strategies:
            self.weights_sum += abs(strategy.weight)
            
            

In [4]:
trader = FuturesTrader(symbol="btcusd", testnet = True)

In [5]:
trader.start_trading(interval = "1m", num_candles = 1000)

2022-12-21 02:58:00.695316: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-21 02:58:00.695427: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-21 02:58:00.695483: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mauricio-SVF14423CLW): /proc/driver/nvidia/version does not exist
2022-12-21 02:58:00.696075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


........................ |pp:0| ................................... |pp:0| ....................................................... |pp:0| ........................................... |pp:0| ................................ |pp:0| .................. |pp:0| ............... |pp:0| .................. |pp:1|  |OP| ....... |pp:0|  |CP| ......... |pp:0| ............ |pp:0| ........... |pp:0| ........................................................ |pp:0| ........................... |pp:1|  |OP| .......... |pp:0|  |CP| ......... |pp:1|  |OP| ..... |pp:1| ...... |pp:1| ..... |pp:1| ... |pp:1| ......... |pp:1| .............. |pp:0|  |CP| ......... |pp:0| ..... |pp:0| .... |pp:0| ..... |pp:0| .. |pp:0| ........... |pp:0| ... |pp:0| ... |pp:0| .... |pp:0| ... |pp:0| ........ |pp:0| ........... |pp:0| ....... |pp:0| ........... |pp:0| ........... |pp:0| .... |pp:0| ....... |pp:0| ..... |pp:1|  |OP| ... |pp:0|  |CP| ............... |pp:0| ............... |pp:0| ....

In [6]:
trader.stop_trading()

In [7]:
trader.data

,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Complete,...,CDL_DRAGONFLYDOJI,CDL_GRAVESTONEDOJI,Close_EBSW,Close_High_Low_ADX_14,Close_High_Low_DMP_14,Close_High_Low_DMN_14,Close_High_Low_Volume_KVO_34_55_13,Close_High_Low_Volume_KVOs_34_55_13,Close_Volume_OBV,rnn
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-20 16:13:00,16885.8,16887.5,16850.0,16882.5,128.154,2.163322e+06,57.0,65.359,1.103570e+06,True,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,128.154,NaN
2022-12-20 16:14:00,16877.8,16884.2,16823.1,16884.2,94.619,1.595140e+06,69.0,36.670,6.190948e+05,True,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,222.773,NaN
2022-12-20 16:15:00,16880.6,16884.2,16843.0,16858.6,264.364,4.460045e+06,69.0,96.636,1.629677e+06,True,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-41.591,NaN
2022-12-20 16:16:00,16855.5,16858.6,16831.0,16843.2,173.829,2.928274e+06,55.0,34.589,5.825719e+05,True,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-215.42,NaN
2022-12-20 16:17:00,16843.2,16843.2,16810.1,16840.6,301.921,5.078801e+06,86.0,79.473,1.337862e+06,True,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-517.341,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21 08:50:00,16826.8,16833.5,16826.8,16831.0,75.570,1.271802e+06,35.0,56.797,9.559127e+05,True,...,0.0,0.0,-0.957146,58.505502,35.564407,13.855573,4.001269,-0.718306,-1351.004,-0.162129
2022-12-21 08:51:00,16832.7,16834.7,16826.8,16826.8,54.801,9.222214e+05,35.0,14.851,2.499874e+05,True,...,0.0,0.0,-0.966537,57.518987,34.684741,13.257361,2.987478,-0.188908,-1405.805,0.210182
2022-12-21 08:52:00,16833.4,16852.3,16833.4,16852.3,8226.000,4.884164e+01,336.0,8182.000,4.858045e+01,True,...,0.0,0.0,0.496989,57.304834,39.165327,11.52732,179.505371,25.481677,6820.195,0.346979


In [ ]:
trader.data.to_csv("data/main.csv")

In [7]:
trader.asset

'USDT'

In [6]:
trader.get_position()

1

..candle completed.........candle completed.......candle completed....candle completed.

### cerca de las 12 horas dio este error:
Unhandled Error
Traceback (most recent call last):
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 96, in callWithLogger
    return callWithContext({"system": lp}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 80, in callWithContext
    return context.call({ILogContext: newCtx}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 117, in callWithContext
    return self.currentContext().callWithContext(ctx, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 82, in callWithContext
    return func(*args, **kw)
--- <exception caught here> ---
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py", line 683, in _doReadOrWrite
    why = selectable.doRead()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py", line 248, in doRead
    return self._dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py", line 253, in _dataReceived
    rval = self.protocol.dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py", line 330, in dataReceived
    self._flushReceiveBIO()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py", line 296, in _flushReceiveBIO
    ProtocolWrapper.dataReceived(self, bytes)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/policies.py", line 110, in dataReceived
    self.wrappedProtocol.dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 348, in dataReceived
    self._dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1243, in _dataReceived
    self.consumeData()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1255, in consumeData
    while self.processData() and self.state != WebSocketProtocol.STATE_CLOSED:
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1619, in processData
    fr = self.onFrameEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1747, in onFrameEnd
    self._onMessageEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 384, in _onMessageEnd
    self.onMessageEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 647, in onMessageEnd
    self._onMessage(payload, self.message_is_binary)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 387, in _onMessage
    self.onMessage(payload, isBinary)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/websocket/binance_client_protocol.py", line 30, in onMessage
    self.factory.callback(payload_obj)
  File "/tmp/ipykernel_16254/1118553294.py", line 52, in message_handler
    self.run_strategy()
  File "/tmp/ipykernel_16254/1118553294.py", line 182, in run_strategy
    self.go_short(prc = True, amount = 95) # go short with full amount
  File "/tmp/ipykernel_16254/1118553294.py", line 201, in go_short
    self.go_neutral() #if some position, go neutral first
  File "/tmp/ipykernel_16254/1118553294.py", line 214, in go_neutral
    self.create_order(side = "SELL", close_pos = True)
  File "/tmp/ipykernel_16254/1118553294.py", line 222, in create_order
    order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 5985, in futures_create_order
    return self._request_futures_api('post', 'order', True, data=params)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 339, in _request_futures_api
    return self._request(method, uri, signed, True, **kwargs)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 315, in _request
    return self._handle_response(self.response)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 324, in _handle_response
    raise BinanceAPIException(response, response.status_code, response.text)
binance.exceptions.BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

WARNING:root:WebSocket connection closed: connection was closed uncleanly ("peer dropped the TCP connection without previous WebSocket closing handshake"), code: 1006, clean: False, reason: connection was closed uncleanly ("peer dropped the TCP connection without previous WebSocket closing handshake")
ERROR:root:Lost connection to Server. Reason: [Failure instance: Traceback: <class 'binance.exceptions.BinanceAPIException'>: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py:80:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py:117:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py:82:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py:696:_doReadOrWrite
--- <exception caught here> ---
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py:683:_doReadOrWrite
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py:248:doRead
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py:253:_dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py:330:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py:296:_flushReceiveBIO
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/policies.py:110:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:348:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1243:_dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1255:consumeData
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1619:processData
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1747:onFrameEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:384:_onMessageEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:647:onMessageEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:387:_onMessage
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/websocket/binance_client_protocol.py:30:onMessage
/tmp/ipykernel_16254/1118553294.py:52:message_handler
/tmp/ipykernel_16254/1118553294.py:182:run_strategy
/tmp/ipykernel_16254/1118553294.py:201:go_short
/tmp/ipykernel_16254/1118553294.py:214:go_neutral
/tmp/ipykernel_16254/1118553294.py:222:create_order
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:5985:futures_create_order
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:339:_request_futures_api
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:315:_request
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:324:_handle_response
]. Retrying: 1